In [ ]:
!pip install pandas
!pip install pyspark
!pip install pyarrow

In [ ]:
from pyspark.sql import SparkSession
 
# Building the SparkSession and name 
# it :'pandas to spark'
spark = SparkSession.builder.appName(
  "pandas to spark").getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")

In [3]:
import pandas as pd

data = [[1, 'Jhon', '2019-01-01', 100], [2, 'Daniel', '2019-01-02', 110], [3, 'Jade', '2019-01-03', 120], [4, 'Khaled', '2019-01-04', 130], [5, 'Winston', '2019-01-05', 110], [6, 'Elvis', '2019-01-06', 140], [7, 'Anna', '2019-01-07', 150], [8, 'Maria', '2019-01-08', 80], [9, 'Jaze', '2019-01-09', 110], [1, 'Jhon', '2019-01-10', 130], [3, 'Jade', '2019-01-10', 150]]
customer = pd.DataFrame(data, columns=['customer_id', 'name', 'visited_on', 'amount']).astype({'customer_id':'Int64', 'name':'object', 'visited_on':'datetime64[ns]', 'amount':'Int64'})

In [4]:
customer = spark.createDataFrame(customer)
customer.show()

+-----------+-------+-------------------+------+
|customer_id|   name|         visited_on|amount|
+-----------+-------+-------------------+------+
|          1|   Jhon|2019-01-01 00:00:00|   100|
|          2| Daniel|2019-01-02 00:00:00|   110|
|          3|   Jade|2019-01-03 00:00:00|   120|
|          4| Khaled|2019-01-04 00:00:00|   130|
|          5|Winston|2019-01-05 00:00:00|   110|
|          6|  Elvis|2019-01-06 00:00:00|   140|
|          7|   Anna|2019-01-07 00:00:00|   150|
|          8|  Maria|2019-01-08 00:00:00|    80|
|          9|   Jaze|2019-01-09 00:00:00|   110|
|          1|   Jhon|2019-01-10 00:00:00|   130|
|          3|   Jade|2019-01-10 00:00:00|   150|
+-----------+-------+-------------------+------+



In [33]:
from pyspark.sql.functions import col, sum, round, max
from datetime import timedelta

customer \
.select('visited_on') \
.distinct() \
.join(
    customer \
    .withColumnRenamed('visited_on', 'day') \
    .select(['day', 'amount']),
    None,
    'cross'
) \
.filter((col('day') >= col('visited_on')) & (col('day') <= col('visited_on') + timedelta(6))) \
.groupby('visited_on') \
.agg(sum('amount').alias('amount')) \
.withColumn('average_amount', round(col('amount') / 7, 2)) \
.withColumn('visited_on', col('visited_on') + timedelta(6)) \
.sort('visited_on') \
.filter(col('visited_on') <= customer.agg(max('visited_on').alias('visited_on')).toPandas()['visited_on'][0]) \
.show()

+-------------------+------+--------------+
|         visited_on|amount|average_amount|
+-------------------+------+--------------+
|2019-01-07 00:00:00|   860|        122.86|
|2019-01-08 00:00:00|   840|         120.0|
|2019-01-09 00:00:00|   840|         120.0|
|2019-01-10 00:00:00|  1000|        142.86|
+-------------------+------+--------------+

